<a href="https://colab.research.google.com/github/JyotiKuber/CodeDivisionProject/blob/main/Bus_Data_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bus Journey to find how may hours each bus is on route V1 - Calculating 5 minutes data in one hour**


In terms of the bus data, a question that I would like answer is how many hours are buses which have Euro III engines (over 15 years old) on the road each day when data has been collected. The data is collected every Wednesday and has a record for every 5 minute interval from 6am to 11.55pm. In each 5 minute interval there is data on every bus out on a route.

First version here is to calculate how many hours each bus is on route (assuming that if it is recorded in one 5 minute interval then 
we can assume that it was on the route for the whole of that 5 minutes) for 1 hour in given day 04-Jan-2023 at 11:05 pm.

Then, using a list of all the buses in the dataset, which has then been updated with the engine type, and then to filter for just the Euro III buses and 
calculate the total number of hours.

## **Dataset Description **



The data for bus journeys is in this folder on Github:

python-programming-for-data/Datasets at main · futureCodersSE/python-programming-for-data

The folder to get the individual links for each file. 

https://raw.githubusercontent.com/futureCodersSE/python-programming-for-data/main/Datasets/bus-journey-data-23-01-04/busdata_23-01-04_23-05.json


and used python to generate links to the files as the link will be the same right up to the filename at the end (ie the file for 23-05)

In [1]:
import requests
import json
import pandas as pd

url= "https://raw.githubusercontent.com/futureCodersSE/python-programming-for-data/main/Datasets/bus-journey-data-23-01-04/busdata_23-01-04_23-05.json"
bus_data = requests.get(url).json()
file_name = url.split('/')[-1]
file_name = file_name.split('.')[0]
recording_date = file_name.split('_')[1]
recording_time = file_name.split('_')[2]
display(recording_date)
display(recording_time)

'23-01-04'

'23-05'

# **Data Cleaning**

RecordedAtTime 2023-01-04T16:31:47+00:00

ItemIdentifier e9de3429-97b4-40c2-9562-a078fe02c673

ValidUntilTime 2023-01-04T23:10:25.372229

MonitoredVehicleJourney {'LineRef': '82', 'DirectionRef': 'inbound', 'FramedVehicleJourneyRef': {'DataFrameRef': '2023-01-04', 'DatedVehicleJourneyRef': '1103'}, 'PublishedLineName': '82', 'OperatorRef': 'AKSS', 'OriginRef': '2400A001090A', 'OriginName': 'Parade', 'DestinationRef': '2400A000640A', 'DestinationName': 'Royal_Star_Arcade', 'OriginAimedDepartureTime': '2023-01-04T16:05:00+00:00', 'DestinationAimedArrivalTime': '2023-01-04T16:37:00+00:00', 'VehicleLocation': {'Longitude': '0.520889', 'Latitude': '51.272936'}, 'BlockRef': '0311', 'VehicleRef': '4295'}

Extensions {'VehicleJourney': {'Operational': {'TicketMachine': {'TicketMachineServiceCode': '82', 'JourneyCode': '1605'}}, 'VehicleUniqueId': '4295'}}


I installed JSON Formatter in Google chrome browser(from chrome webstore). Now after restarting Chrome, I can see parsed JSON which is more readable to work.

In [ ]:
for item in bus_data:

        print(item['RecordedAtTime'])
        vehicle_journey_data = item['MonitoredVehicleJourney']
        journey_keys_list = list(vehicle_journey_data.keys())

        for key in journey_keys_list:
            print( key, "-",vehicle_journey_data[key])

        extensions_data = item['Extensions']
        extensions_keys_list = list(extensions_data.keys())
        for key1 in extensions_keys_list:
            print( key1, "-",extensions_data[key1])



2023-01-04T16:31:47+00:00
LineRef - 82
DirectionRef - inbound
FramedVehicleJourneyRef - {'DataFrameRef': '2023-01-04', 'DatedVehicleJourneyRef': '1103'}
PublishedLineName - 82
OperatorRef - AKSS
OriginRef - 2400A001090A
OriginName - Parade
DestinationRef - 2400A000640A
DestinationName - Royal_Star_Arcade
OriginAimedDepartureTime - 2023-01-04T16:05:00+00:00
DestinationAimedArrivalTime - 2023-01-04T16:37:00+00:00
VehicleLocation - {'Longitude': '0.520889', 'Latitude': '51.272936'}
BlockRef - 0311
VehicleRef - 4295
VehicleJourney - {'Operational': {'TicketMachine': {'TicketMachineServiceCode': '82', 'JourneyCode': '1605'}}, 'VehicleUniqueId': '4295'}
2023-01-04T16:01:58+00:00
LineRef - 693
DirectionRef - inbound
FramedVehicleJourneyRef - {'DataFrameRef': '2023-01-04', 'DatedVehicleJourneyRef': '1001'}
PublishedLineName - 693
OperatorRef - AKSS
OriginRef - 249000000309
OriginName - Breton_Road
DestinationRef - 249000000171
DestinationName - Linwood_Avenue
OriginAimedDepartureTime - 2023-01

Initial processing requirements - create a dataframe from the tracked buses data that has the following columns:
- RecordedAtTime
- LineRef
- OperatorRef
- VehicleRef

Here as the data contains dates and time from different days also first we'll need to clean the data using the date on JSON file and it should match to the RecordedAtTime in the file records.

Here we are collecting only the records which are in given date 04/01/2023 and time 23:05. The timestamp is used to filter out the bus route information between 5 minutes.

Once we get the number of EURO III engine buses running on the road in 5 minutes we can calculate the buses in one hour and then in one day.

Save filtered data in an Excel or CSV file.

In [2]:
from datetime import datetime, timedelta
def get_snapshot_time_from_file_name(fname):
  file_name = fname.split('/')[-1]
  file_name = file_name.split('.')[0]
  recording_date = '20' + file_name.split('_')[1]
  recording_time = file_name.split('_')[2]
  required_day = int(recording_date.split('-')[2])
  required_year = int(recording_date.split('-')[0])
  required_month= int(recording_date.split('-')[1])
  required_hour = int(recording_time.split('-')[0])
  required_minutes= int(recording_time.split('-')[1])
  snapshot_time = datetime(required_year,required_month,required_day,required_hour,required_minutes)
  return snapshot_time

def in_this_5_mins(RecordedAtTime, snapshot_time):
  format = '%Y-%m-%dT%H:%M:%S+%f:00'
  # converting the timestamp string to datetime object
  datetime_object = datetime.strptime(RecordedAtTime, format)
  return snapshot_time.timestamp() - datetime_object.timestamp() < 300

vehicle_list = []
snapshot_time = get_snapshot_time_from_file_name(file_name)
for item in bus_data:
  RecordedAtTime = item['RecordedAtTime']
  if in_this_5_mins(RecordedAtTime, snapshot_time):
    print(RecordedAtTime, snapshot_time)
    vehicle_journey_data = item['MonitoredVehicleJourney']
    LineRef = vehicle_journey_data['LineRef']
    OperatorRef = vehicle_journey_data['OperatorRef']
    VehicleRef= vehicle_journey_data['VehicleRef']
    my_dict = {'RecordedAtTime': RecordedAtTime,'LineRef': LineRef,'OperatorRef': OperatorRef, 'VehicleRef': VehicleRef}
    vehicle_list.append(my_dict)
vehicle1_df = pd.DataFrame(vehicle_list)
print(vehicle1_df)
print(vehicle1_df.info())

2023-01-04T23:04:59+00:00 2023-01-04 23:05:00
2023-01-04T23:04:33+00:00 2023-01-04 23:05:00
2023-01-04T23:04:53+00:00 2023-01-04 23:05:00
2023-01-04T23:02:08+00:00 2023-01-04 23:05:00
2023-01-04T23:05:00+00:00 2023-01-04 23:05:00
2023-01-04T23:04:43+00:00 2023-01-04 23:05:00
2023-01-04T23:04:49+00:00 2023-01-04 23:05:00
2023-01-04T23:04:56+00:00 2023-01-04 23:05:00
2023-01-04T23:04:59+00:00 2023-01-04 23:05:00
2023-01-04T23:05:13+00:00 2023-01-04 23:05:00
2023-01-04T23:05:10+00:00 2023-01-04 23:05:00
2023-01-04T23:04:50+00:00 2023-01-04 23:05:00
2023-01-04T23:05:11+00:00 2023-01-04 23:05:00
2023-01-04T23:04:57+00:00 2023-01-04 23:05:00
               RecordedAtTime LineRef OperatorRef VehicleRef
0   2023-01-04T23:04:59+00:00     101        AKSS       6412
1   2023-01-04T23:04:33+00:00       3        AKSS       6138
2   2023-01-04T23:04:53+00:00     166        AKSS       4313
3   2023-01-04T23:02:08+00:00     101        AKSS       4273
4   2023-01-04T23:05:00+00:00     191        AKSS  

There is an updated dataset here with the engine types for each vehicle (by vehicleRef). python-programming-for-data/updated_bus_regs.csv at main · futureCodersSE/python-programming-for-data

Your next step will be to create a copy of your dataframe, with an extra column 'Emission Class' and to fill in the value for each row of data by looking up the vehicle ref and getting the emission class.

This will result in a new dataframe with some null values in the Emission Class column as the list may not be completely up to date. It looks like this might be a fairly up to date version but we need to check. However, we might be able to find a trend in the vehicle refs (e.g. if the vehicle ref is greater than 6000 then the vehicle is probably a EURO VI - this may not be the case but it is worth investigating).

In [3]:
def get_emission_data():
  url = "https://raw.githubusercontent.com/futureCodersSE/python-programming-for-data/main/Datasets/updated_bus_regs.csv"
  df = pd.read_csv(url)
  return df

def add_emission_class(v_df):
  emissions_df = get_emission_data()

  def get_emission_class(df):
      em_list = emissions_df[emissions_df['Last tracked'] == df['VehicleRef']]['Emission Class'].tolist()
        
      if len(em_list) > 0:
         return em_list[0]
      else:
         return None

  v_df['Emission Class'] = v_df.apply(get_emission_class, axis=1)
  return v_df
# Testing
em_df = vehicle1_df.copy()
em_df['VehicleRef'] = pd.to_numeric(em_df['VehicleRef'], errors="coerce")
actual = add_emission_class(em_df)
display(actual)

,RecordedAtTime,LineRef,OperatorRef,VehicleRef,Emission Class
0,2023-01-04T23:04:59+00:00,101,AKSS,6412,EURO III
1,2023-01-04T23:04:33+00:00,3,AKSS,6138,EURO III
2,2023-01-04T23:04:53+00:00,166,AKSS,4313,EURO VI
3,2023-01-04T23:02:08+00:00,101,AKSS,4273,EURO VI
4,2023-01-04T23:05:00+00:00,191,AKSS,6428,EURO III
5,2023-01-04T23:04:43+00:00,5,AKSS,4056,EURO IV
6,2023-01-04T23:04:49+00:00,12,AKSS,4066,EURO IV
7,2023-01-04T23:04:56+00:00,7,AKSS,4126,EURO VI
8,2023-01-04T23:04:59+00:00,132,AKSS,1651,EURO V
9,2023-01-04T23:05:13+00:00,177,AKSS,4058,EURO IV


In [11]:
emission_copy = actual.groupby(['Emission Class']).count()
display(emission_copy)
euro_3_df = actual[actual['Emission Class'] == 'EURO III']
print("Unique number of EURO III buses on the road:",len(euro_3_df['VehicleRef'].unique()))
# Calculate EURO III vehicles in 5 minutes time
euro_3_vehicles = emission_copy['VehicleRef']['EURO III']
euro_3_vehicles_5minutes = euro_3_vehicles * 5
print("Total number of on road hours for EURO III buses:",euro_3_vehicles_5minutes) # shows number of EURO III vehicles run on the road in 5 minutes

,RecordedAtTime,LineRef,OperatorRef,VehicleRef
Emission Class,,,,
EURO III,4,4,4,4
EURO IV,3,3,3,3
EURO V,1,1,1,1
EURO VI,5,5,5,5


Unique number of EURO III buses on the road: 4
Total number of on road hours for EURO III buses: 20


Mounting and unmounting the Google Drive
In order to be able to open and save files on Google Drive, with Python, you first need to mount the Drive.

In [ ]:
from google.colab import drive

def mount_drive():
  drive.mount('/content/drive', force_remount=True)
  folder_name = "/content/drive/MyDrive/Colab_data"
  return folder_name

data_folder = mount_drive()
print(data_folder)

Mounted at /content/drive
/content/drive/MyDrive/Colab_data


The code will save a copy of the file as vehicle_data.csv

In [ ]:
def save_data(df, path):
  try:
    df.to_csv(path + "/vehicle_data.csv")
    print("File saved successfully")
  except:
    print("There was an error when trying to save the file.")
path = mount_drive()
save_data(vehicle1_df, path)

Mounted at /content/drive
File saved successfully


In [ ]:
from pandas.io.formats.info import DataFrameInfo
def save_data(df, path):
  try:
    df.to_csv(path + "/vehicle_emission_data.csv")
    print("File saved successfully")
  except:
    print("There was an error when trying to save the file.")
path = mount_drive()
save_data(actual, path)

After finished working with the files, you should always unmount the Drive.

In [ ]:
def unmount_drive():
  drive.flush_and_unmount()
  print('All changes made in this colab session should now be visible in Drive.')

unmount_drive()

All changes made in this colab session should now be visible in Drive.


From above results we can see there are 20 EURO III buses run on the road in 5 minutes and 240 EURO III engine buses in one hour.